# Proyecto ETL: Datos de Bicicletas Públicas de Londres

## Descripción
Este proyecto implementa un flujo ETL (Extract, Transform, Load) que obtiene datos en tiempo real de las estaciones de bicicletas públicas de Londres a través de la API de Transport for London (TfL).

## Tecnologías utilizadas
- **Python 3.x**
- **Pandas**: Manipulación y análisis de datos
- **Requests**: Consumo de API REST
- **PyArrow**: Almacenamiento en formato Parquet

## Estructura del proyecto
```
flujo_etl/
├── etl_api_transporte_londres.ipynb  # Notebook principal
├── datos_transporte_londres.parquet  # Datos procesados
├── requirements.txt                   # Dependencias
├── .env                               # Variables de entorno (no subir a git)
└── README.md                          # Documentación
```

In [1]:
# Librerías a usar
import requests
import os
import logging
from datetime import datetime
from typing import Optional
from dotenv import load_dotenv
import pandas as pd

In [3]:
# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)

# Configuración
class Config:
    """Clase de configuración para el proyecto ETL."""

    ENDPOINT = "https://api.tfl.gov.uk/BikePoint/"
    OUTPUT_FILE = "datos_transporte_londres.parquet"
    REQUEST_TIMEOUT = 30  # segundos

    def __init__(self):
        load_dotenv()
        self.app_id = os.getenv("APP_ID")
        self.primary_key = os.getenv("PRIMARY_KEY")

    def validate(self) -> bool:
        """Valida que las credenciales estén configuradas."""
        if not self.app_id or not self.primary_key:
            logger.warning("Credenciales no configuradas. La API puede limitar las solicitudes.")
            return False
        return True

config = Config()
config.validate()

True

## Fase E (Extraer)

En esta fase accedemos a la API de TfL y descargamos los datos JSON de las estaciones de bicicletas.

In [4]:
def extract_data(config: Config) -> Optional[list]:
    """
    Extrae datos de la API de Transport for London.

    Args:
        config: Objeto de configuración con credenciales y endpoint.

    Returns:
        Lista de estaciones en formato JSON o None si hay error.

    Raises:
        requests.exceptions.RequestException: Si hay un error en la petición.
    """
    params = {
        "app_id": config.app_id,
        "app_key": config.primary_key
    }

    try:
        logger.info(f"Conectando a la API: {config.ENDPOINT}")
        response = requests.get(
            config.ENDPOINT,
            params=params,
            timeout=config.REQUEST_TIMEOUT
        )
        response.raise_for_status()

        data = response.json()
        logger.info(f"Se descargaron datos de {len(data)} estaciones")
        return data

    except requests.exceptions.Timeout:
        logger.error("Timeout: La API tardó demasiado en responder")
        raise
    except requests.exceptions.HTTPError as e:
        logger.error(f"Error HTTP: {e.response.status_code} - {e.response.reason}")
        raise
    except requests.exceptions.RequestException as e:
        logger.error(f"Error de conexión: {e}")
        raise

# Ejecutar extracción
data = extract_data(config)

2026-02-11 19:52:13,645 - INFO - Conectando a la API: https://api.tfl.gov.uk/BikePoint/
2026-02-11 19:52:16,436 - INFO - Se descargaron datos de 801 estaciones


### Exploración de la estructura de datos

Veamos la estructura de los datos recibidos para entender cómo transformarlos.

In [5]:
# Explorar estructura de una estación
if data:
    print(f"Total de estaciones: {len(data)}")
    print(f"\nEjemplo de estación (primera):")
    print(f"  - Nombre: {data[0]['commonName']}")
    print(f"  - Latitud: {data[0]['lat']}")
    print(f"  - Longitud: {data[0]['lon']}")
    print(f"\nCampos adicionales disponibles:")
    for item in data[0]['additionalProperties']:
        print(f"  - {item['key']}: {item['value']}")

Total de estaciones: 801

Ejemplo de estación (primera):
  - Nombre: River Street , Clerkenwell
  - Latitud: 51.529163
  - Longitud: -0.10997

Campos adicionales disponibles:
  - TerminalName: 001023
  - Installed: true
  - Locked: false
  - InstallDate: 1278947280000
  - RemovalDate: 
  - Temporary: false
  - NbBikes: 3
  - NbEmptyDocks: 11
  - NbDocks: 19
  - NbStandardBikes: 2
  - NbEBikes: 1


## Fase T (Transformar)

Transformamos los datos JSON a un formato tabular estructurado, extrayendo solo los campos relevantes.

In [6]:
def get_additional_property(properties: list, key: str, default=None):
    """
    Obtiene un valor de additionalProperties de forma segura.

    Args:
        properties: Lista de propiedades adicionales.
        key: Clave a buscar.
        default: Valor por defecto si no se encuentra.

    Returns:
        El valor encontrado o el valor por defecto.
    """
    for item in properties:
        if item.get('key') == key:
            return item.get('value', default)
    return default


def transform_station(station: dict) -> dict:
    """
    Transforma los datos de una estación individual.

    Args:
        station: Diccionario con datos de la estación.

    Returns:
        Diccionario con datos transformados.
    """
    props = station.get('additionalProperties', [])

    # Convertir 'Locked' a disponibilidad
    locked = get_additional_property(props, 'Locked', 'true')
    disponible = 'Sí' if locked.lower() == 'false' else 'No'

    return {
        'nombre_estacion': station.get('commonName', 'Desconocido'),
        'disponible': disponible,
        'n_bicis': int(get_additional_property(props, 'NbBikes', 0)),
        'n_espacios_disponibles': int(get_additional_property(props, 'NbEmptyDocks', 0)),
        'n_espacios_total': int(get_additional_property(props, 'NbDocks', 0)),
        'n_bicis_estandar': int(get_additional_property(props, 'NbStandardBikes', 0)),
        'n_bicis_electricas': int(get_additional_property(props, 'NbEBikes', 0)),
        'latitud': station.get('lat'),
        'longitud': station.get('lon'),
    }


def transform_data(data: list) -> pd.DataFrame:
    """
    Transforma la lista de estaciones a un DataFrame.

    Args:
        data: Lista de estaciones en formato JSON.

    Returns:
        DataFrame con los datos transformados.
    """
    if not data:
        logger.warning("No hay datos para transformar")
        return pd.DataFrame()

    logger.info("Iniciando transformación de datos...")

    # Transformar cada estación
    stations_transformed = [transform_station(station) for station in data]

    # Crear DataFrame
    df = pd.DataFrame(stations_transformed)

    # Agregar metadatos
    df['fecha_extraccion'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    logger.info(f"Transformación completada: {len(df)} registros procesados")
    return df

# Ejecutar transformación
df = transform_data(data)
df.head(10)

# Validación y estadísticas de los datos
def validate_data(df: pd.DataFrame) -> bool:
    """
    Valida la calidad de los datos transformados.

    Args:
        df: DataFrame a validar.

    Returns:
        True si los datos son válidos, False en caso contrario.
    """
    if df.empty:
        logger.error("DataFrame vacío")
        return False

    # Verificar valores nulos
    null_counts = df.isnull().sum()
    if null_counts.any():
        logger.warning(f"Valores nulos encontrados:\n{null_counts[null_counts > 0]}")

    # Estadísticas básicas
    print("\nEstadísticas del Dataset:")
    print(f"   - Total de estaciones: {len(df)}")
    print(f"   - Estaciones disponibles: {(df['disponible'] == 'Sí').sum()}")
    print(f"   - Estaciones no disponibles: {(df['disponible'] == 'No').sum()}")
    print(f"   - Total de bicicletas: {df['n_bicis'].sum():,}")
    print(f"   - Bicicletas eléctricas: {df['n_bicis_electricas'].sum():,}")
    print(f"   - Espacios disponibles: {df['n_espacios_disponibles'].sum():,}")

    return True

validate_data(df)

# Información del DataFrame
print("Información del DataFrame:")
df.info()
print("\nEstadísticas descriptivas:")
df.describe()

2026-02-11 19:53:02,505 - INFO - Iniciando transformación de datos...
2026-02-11 19:53:02,555 - INFO - Transformación completada: 801 registros procesados



Estadísticas del Dataset:
   - Total de estaciones: 801
   - Estaciones disponibles: 801
   - Estaciones no disponibles: 0
   - Total de bicicletas: 9,529
   - Bicicletas eléctricas: 1,193
   - Espacios disponibles: 9,950
Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nombre_estacion         801 non-null    object 
 1   disponible              801 non-null    object 
 2   n_bicis                 801 non-null    int64  
 3   n_espacios_disponibles  801 non-null    int64  
 4   n_espacios_total        801 non-null    int64  
 5   n_bicis_estandar        801 non-null    int64  
 6   n_bicis_electricas      801 non-null    int64  
 7   latitud                 801 non-null    float64
 8   longitud                801 non-null    float64
 9   fecha_extraccion        801 non-null    object 
dtypes: f

,n_bicis,n_espacios_disponibles,n_espacios_total,n_bicis_estandar,n_bicis_electricas,latitud,longitud
count,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000
mean,11.896380,12.421973,26.253433,10.406991,1.489388,51.506083,-0.127214
std,8.895252,9.572576,8.586585,8.238122,1.974259,0.020367,0.055070
min,0.000000,0.000000,5.000000,0.000000,0.000000,51.452997,-0.236769
25%,4.000000,4.000000,20.000000,3.000000,0.000000,51.493146,-0.171185
50%,12.000000,11.000000,24.000000,10.000000,1.000000,51.509230,-0.129361
75%,18.000000,18.000000,30.000000,16.000000,2.000000,51.521113,-0.090847
max,43.000000,56.000000,63.000000,37.000000,14.000000,51.549369,-0.002275


## Fase L (Carga)

Almacenamos los datos procesados en formato Parquet, optimizado para análisis y visualización posterior.

In [10]:
def load_data(df: pd.DataFrame, output_path: str) -> bool:
    """
    Guarda el DataFrame en formato Parquet.

    Args:
        df: DataFrame a guardar.
        output_path: Ruta del archivo de salida.

    Returns:
        True si se guardó correctamente, False en caso contrario.
    """
    if df.empty:
        logger.error("No hay datos para guardar")
        return False

    try:
        df.to_parquet(output_path, engine='pyarrow', index=False)
        file_size = os.path.getsize(output_path) / 1024  # KB
        logger.info(f"Datos guardados en: {output_path} ({file_size:.2f} KB)")
        return True
    except Exception as e:
        logger.error(f"Error al guardar los datos: {e}")
        return False

# Ejecutar carga
load_data(df, config.OUTPUT_FILE)

2026-02-11 19:57:04,291 - INFO - Datos guardados en: datos_transporte_londres.parquet (37.35 KB)


True

## Verificación Final

Verificamos que los datos se hayan guardado correctamente leyendo el archivo Parquet.

In [11]:
# Verificar lectura del archivo
df_verificacion = pd.read_parquet(config.OUTPUT_FILE)
print(f"Verificación exitosa: {len(df_verificacion)} registros leídos")
df_verificacion.head()

Verificación exitosa: 801 registros leídos


,nombre_estacion,disponible,n_bicis,n_espacios_disponibles,n_espacios_total,n_bicis_estandar,n_bicis_electricas,latitud,longitud,fecha_extraccion
0,"River Street , Clerkenwell",Sí,3,11,19,2,1,51.529163,-0.109970,2026-02-11 19:53:02
1,"Phillimore Gardens, Kensington",Sí,7,30,37,5,2,51.499606,-0.197574,2026-02-11 19:53:02
2,"Christopher Street, Liverpool Street",Sí,12,18,32,11,1,51.521283,-0.084605,2026-02-11 19:53:02
3,"St. Chad's Street, King's Cross",Sí,21,1,23,20,1,51.530059,-0.120973,2026-02-11 19:53:02
4,"Sedding Street, Sloane Square",Sí,6,20,27,6,0,51.493130,-0.156876,2026-02-11 19:53:02


## Resumen del Pipeline ETL

| Fase | Descripción | Estado |
|------|-------------|--------|
| **Extract** | Obtener datos de API TfL | ✅ |
| **Transform** | Limpiar y estructurar datos | ✅ |
| **Load** | Guardar en formato Parquet | ✅ |
